In [15]:
import numpy as np
import csv
import pandas as pd

In [16]:
# dedicated runtime
appRuntime = np.load('../dedicated_runtime/app_dedicated_rt.npy').item()

In [17]:
def read_timing_df(curfile):
    """read timing traces in csv file, offset the trace with the starting pos, return the dataframe"""
    df_current = pd.read_csv(curfile)
    offset = df_current['start'].min()  # find the starting timing, and offset it
    df_current['start'] = df_current['start'].apply(lambda x: x - offset)
    df_current['end'] = df_current['end'].apply(lambda x: x - offset)
    return df_current

def compare_with_dedicatedruntime(df, appRuntime):
    """
    read the timing traces in dataframe (after read_timing_df()).
    compute the speedup over the dedicated runtime
    return speedup list
    """
    dedicated_total,  corun_total = 0.0, 0.0
    speedup_list = []

    # columns => ['jobID', 'appName', 'start', 'end', 'duration (s)']
    for index, row in df.iterrows():
        app_name, app_endT = row['appName'], row['end']
        app_rt_dedicate = float(appRuntime[app_name])  # dedicated runtime
        app_endT = float(app_endT)
        if corun_total < app_endT: corun_total = app_endT
        dedicated_total += app_rt_dedicate
        speedup = dedicated_total / corun_total
        speedup_list.append(speedup)   
    return speedup_list

In [18]:
len(appRuntime)

79

In [19]:
appRuntime

{'cudasdk_BlackScholes': '1.4004418849945068',
 'cudasdk_FDTD3d': '14.441001176834106',
 'cudasdk_MCEstimatePiInlineP': '0.39530420303344727',
 'cudasdk_MCEstimatePiInlineQ': '0.3933548927307129',
 'cudasdk_MCEstimatePiP': '0.3939080238342285',
 'cudasdk_MCEstimatePiQ': '0.4072442054748535',
 'cudasdk_MCSingleAsianOptionP': '0.39748215675354004',
 'cudasdk_SobolQRNG': '0.5573899745941162',
 'cudasdk_batchCUBLAS': '0.7025609016418457',
 'cudasdk_binomialOptions': '25.258795976638794',
 'cudasdk_boxFilterNPP': '0.4159379005432129',
 'cudasdk_c++11Cuda': '0.5548751354217529',
 'cudasdk_concurrentKernels': '0.39653706550598145',
 'cudasdk_convolutionFFT2D': '3.801888942718506',
 'cudasdk_convolutionSeparable': '1.9495930671691895',
 'cudasdk_convolutionTexture': '1.1159520149230957',
 'cudasdk_dct8x8': '0.7084078788757324',
 'cudasdk_dwtHaar1D': '0.39554309844970703',
 'cudasdk_dxtc': '0.41524600982666016',
 'cudasdk_eigenvalues': '1.3545660972595215',
 'cudasdk_fastWalshTransform': '3.368

In [20]:
def compute_stp(inputcsv):
    """Read app status table in csv format, compute the ANTT"""
    
    df = read_timing_df(inputcsv)
    
    #
    # 1) sort the dataframe by 'end'
    #
    sorted_df = df.sort_values(by=['end'])
    
    
    #
    # 2) compute the stp for each app
    # 
    stp_list = []
    for idx, row in sorted_df.iterrows():
        appName = row['appName']
        T_corun = float(row['duration (s)'])
        T_dedicate = float(appRuntime[appName])

        tmp =  T_dedicate / T_corun
        stp_list.append(tmp)

    #print stp_list
    
    #
    # 3) in stp list, add current value with prev
    # 
    stp = [0 for i in xrange(len(stp_list))]
    for i, v in enumerate(stp_list):
        if i == 0:
            stp[0] = v
        else:
            stp[i] = stp[i-1] + v
    
    return stp
    

### read traces

In [21]:
inputcsv = "../fcfs_csv/fcfs_run8_s1.csv"
stp = compute_stp(inputcsv)
print stp

mydf = pd.DataFrame(stp)
mydf.to_csv('stp_fcfs_max8.csv', index=False, header=False)

[0.12283269547502497, 0.25398453686081984, 0.49110616254541317, 0.6536391782845794, 0.8486476984558528, 1.1420874940634689, 1.3402197761548122, 1.769046283755994, 2.168047582089999, 2.3959713433971643, 2.6051355893189267, 2.933861207302715, 3.13845781821005, 3.3079242429221076, 3.492620395565043, 3.8867286823127234, 4.118221973530761, 4.744811622107714, 5.181442361793715, 5.511364193206329, 6.045426685434949, 6.365319366976825, 6.713440887096779, 7.0222617512940815, 7.399337249187682, 7.889903402451361, 8.200656127229704, 8.769859170636186, 9.039683440330524, 9.58028927919896, 9.960950193198228, 10.507235645620742, 10.703878250619022, 11.20665351074269, 11.48449704286622, 11.822614229708591, 12.254998856491653, 12.57967098705767, 12.969776837973958, 13.451038936727452, 13.783342566007104, 14.095786791296641, 14.358327381129639, 14.990626772119235, 15.298930209359064, 15.50248761551946, 15.766441902100704, 16.394607996788977, 16.626805434363856, 16.959845965832336, 17.21141979364244, 17

In [23]:
inputcsv = "../sim_csv/feat9/sim_feat9_run8_s1.csv"
stp = compute_stp(inputcsv)
print stp

mydf = pd.DataFrame(stp)
mydf.to_csv('stp_sim-feat9_max8.csv', index=False, header=False)

[0.13206391981389726, 0.2356572499526202, 0.4434928312804695, 0.7102430943599206, 0.9184539010495023, 1.1123968119537029, 1.504685710388674, 1.8357702518772478, 2.1822070153858104, 2.498205079106596, 2.7611839763715746, 3.051600540650365, 3.354549144923388, 3.6368745262462827, 4.024695836656119, 4.18479963474711, 4.465423162282855, 4.705801029254804, 4.819956139275379, 4.94899236988815, 5.289559647865758, 5.524839939262458, 5.726229407175523, 5.981756590582184, 6.20181553487343, 6.409491158841145, 6.620712200761866, 6.768619180508242, 6.978347081271062, 7.10545551527651, 7.270111410589593, 7.448410213833426, 7.661621108786233, 7.936041007700461, 8.144173122608608, 8.343213114416928, 8.562768967655995, 8.78400820602495, 9.0088273675243, 9.376386504559143, 9.673951689847271, 10.052320456030735, 10.317478488540969, 10.571850366257639, 10.914165759908773, 11.195480551553583, 11.568702727305478, 11.826487437092203, 12.175493578713072, 12.55028490489111, 12.931315126574216, 13.23850543319954

In [24]:
inputcsv = "../robustBin_csv/feat9/robustBin_feat9_run8_s1.csv"
stp = compute_stp(inputcsv)
print stp

mydf = pd.DataFrame(stp)
mydf.to_csv('stp_interBin-feat9_max8.csv', index=False, header=False)

[0.20023032159749174, 0.4383117524312443, 0.7728392991683065, 1.2224919888745922, 1.5593519566260607, 2.120045419950509, 2.4460418695440813, 2.9996117935811206, 3.283623551357461, 3.9628703730492463, 4.288039119204376, 4.6817600080420485, 5.025931442167361, 5.486325258541618, 5.878018549188167, 6.224521558329333, 6.606148819978797, 6.918681989443337, 7.218953334136215, 7.575168490555936, 7.8663341449917095, 8.26227770431627, 8.59352546536395, 8.992110858561375, 9.51056953444109, 9.832122894855512, 10.438613758393892, 10.786897729341119, 11.140233131914785, 11.730079472232756, 12.171227501928698, 12.521294574036988, 12.883276900414259, 13.2316829688765, 13.727919253027396, 14.35682592921897, 14.792350763723295, 15.207991775388853, 15.605670765702964, 16.0268937085444, 16.48291706594378, 17.239800373357433, 17.774189862106812, 18.314986696365576, 18.670168082356202, 18.95428856800091, 19.22302338773969, 19.46751540997733, 20.0104288206215, 20.665289453330704, 21.03358655622621, 21.603412